# Blockchain analysis project

## Aim of the project
Blockchain projects using a crypto are flourishing today, it has become difficult to assess rapidly the quality of a new project. We will try to combine various factors to help us dig in a project such as : price, stock market cap, Twitter sentiment analysis, financial analysis, CoinGecko KPIs (Ratings, sentiment, n° of exchanges available).

## 1. Retrieve CoinGecko's data

In [1]:
import pandas as pd
from selenium import webdriver
import tweepy

In [23]:
# Function to extract the number of likes in the sentence "xxx personnes aiment ça"
# Input : string, extracted from Coingecko's website
# Output : Number of likes (int)

def extract_int_from_sentence(sentence):
    numbers = [int(s) for s in sentence.split() if s.isdigit()]
    figure_string = ''.join(map(str, numbers))
    figure = int(figure_string)
    return figure

In [33]:
# Function to clean the price output of coingecko
# Input : unformated price (string)
# Output : price (int)

def format_price(price_string):
    price_string = price_string.replace("\u202f","")
    price_string = price_string.replace(" $US","")
    price_string = price_string.replace(",", ".")
    price = float(price_string)
    return price

In [31]:
# Function that returns coin gecko's data about a given coin
# Input : full coin name (ex: bitcoin, ethereum...) (string)
# Output : Number of favorites (int), current price in $ (float), 

def retrieve_coingecko_data(coin_name):
    
    driver = webdriver.Chrome('/Users/manuel/Documents/GitHub/chromedriver')
    driver.get(f"https://www.coingecko.com/fr/pi%C3%A8ces/{coin_name}#markets")
    driver.implicitly_wait(10)
    
    likes_string = driver.find_element_by_xpath('/html/body/div[4]/div[4]/div[1]/div/div[4]/div/div[4]/div/div[4]').text
    nb_likes = extract_int_from_sentence(likes_string)
    
    price_string = driver.find_element_by_xpath('/html/body/div[4]/div[4]/div[1]/div/div[4]/div/div[1]/span[1]/span').text
    price = format_price(price_string)
    
    daily_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[4]/div[1]/div/div[4]/div/div[1]/span[2]/span').text
    daily_evolution = round(float(daily_evolution_string.replace('%',''))/100,3)
    
    market_cap_string = driver.find_element_by_xpath('/html/body/div[4]/div[5]/div[1]/div/div[1]/div/div/div[2]/span[2]/span').text
    market_cap = format_price(market_cap_string)
    
    daily_trading_string = driver.find_element_by_xpath('/html/body/div[4]/div[5]/div[1]/div/div[1]/div/div/div[3]/span[2]/span').text
    daily_trading = format_price(daily_trading_string)
    
    
    return ({"nb_likes":nb_likes,
             "price ($US)":price,
             "24h evolution":daily_evolution,
             "market cap":market_cap,
             "daily trading volume":daily_trading})

In [32]:
retrieve_coingecko_data("ethereum")

{'nb_likes': 556740,
 'price ($US)': 2626.0,
 '24h evolution': 0.041,
 'market cap': 306878676055.0,
 'daily trading volume': 23715372320.0}

# Twitter sentiment analysis

In [2]:
consumer_key = "7S41BVFwVu7wUhHV0csj41KK5"
consumer_secret = "Vjsfv9ecbWrLQFo7X74kZZY2XRnhokfsajq1QcPq2uaSzYvdNT"
access_token = "606608726-mImNVp0hfEt9MDo9c2S4zNBZU9lS1z4LqwkDT6oY"
access_token_secret = "jR7c83KQVD3VwRtik7pV30ATaY6tzDyFgo11xD6zN30F4"

# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# set access to user's access key and access secret  
auth.set_access_token(access_token, access_token_secret) 
  
# calling the api  
api = tweepy.API(auth) 